In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import matplotlib.pyplot as plt
from scipy import stats
from scipy import cluster

In [2]:
n = 1000
beta01, beta11 = 5,-3
beta02, beta12 = 2, 4

#set up regression mixture
x1 = np.random.uniform(0, 10, size=400)
x2 = np.random.uniform(0, 10, size=600)

y1 = beta01 + beta11*x1 + np.random.normal(scale=5.0, size=400)
y2 = beta02 + beta12*x2 + np.random.normal(scale=4.0,size=600)

X = np.concatenate([x1, x2])
Y = np.concatenate([y1, y2])


#set up 2 component mixture
a1 = np.random.normal(2, 5, size=600)
a2 = np.random.normal(5, 3, size=400)
a = np.concatenate([a1,a2])

In [3]:
def e_step(y,x,params): 
    y, x = np.array(y), np.array(x)
    nobs, k = x.shape
    weights = []
    for param in params:

        sigma = param[-1]
        beta = np.tile(param[1:-1],nobs).reshape(nobs, k)
        mean = (beta*x).sum(axis=1)
        weights.append( stats.norm.pdf(y, loc=mean, scale=sigma)*param[0] )
        
    #update loop variables
    weights = np.array(weights).transpose()
    #denom = np.repeat( 1+np.exp(weights).sum(axis=1), len(params) ).reshape(nobs,len(params))
    denom = np.repeat(weights.sum(axis=1), len(params) ).reshape(nobs,len(params))
    weights = weights/denom
    return weights, np.log(denom[:,0])
        
    
def m_step(y,x,weights):
    y, x, weights = np.array(y), np.array(x), np.array(weights)
    nobs, k = x.shape
    params, se, err = [], [], 0

    for w in weights.transpose():
        
        lamb = w.mean()
        lamb_se = w.std()

        #beta
        w_mat = np.diag(w)
        xx_mat = np.linalg.inv( x.transpose().dot( w_mat).dot(x) )
        beta = xx_mat.dot(x.transpose().dot(w_mat)).dot(y)
        
        #sigma
        mu = np.tile(beta, nobs).reshape(nobs, k)*x
        weighted_err = w*(y - mu.sum(axis=1))**2
        sigma =  (weighted_err.sum()/w.sum())**.5

        #add component
        comp_param =np.concatenate(([lamb],beta,[sigma]))
        params.append(comp_param)

        #beta_se
        beta_se = (np.diagonal(xx_mat*sigma**2))**.5
        comp_se = np.concatenate(([lamb_se],beta_se))
        se.append(comp_se)

        #SSR
        err = err+weighted_err
    return np.array(params), np.array(se), 1-err.mean()/y.var()


def gen_weights(y,ncomp):
    c,labels = cluster.vq.kmeans2(y,ncomp)
    return np.array(pd.get_dummies(labels))


def estimate(y,x,ncomp):
    e = gen_weights(y,ncomp)
    m = None
    for i in range(20):
        m,se,r2 = m_step(y,x,e)
        e,ll = e_step(y,x,m)
    return m, se, r2, y, x, ncomp, ll


m, se, r2, y, x, ncomp, ll = estimate(Y, sm.add_constant(X), 1)

In [4]:
def write_table(fname, estimates, labels=('y',None)):
    
    #unpack relevant information
    params, se, r2, y, x, ncomp, ll = estimates
    nobs, k = x.shape
    ylabel, xlabel = labels
    
    #calc aic
    aic = 2*(params.shape[0]*params.shape[1]-2) - 2*ll.sum()
    aic = np.round(aic,1)
    
    if xlabel == None:
        xlabel =[]
        for i in range(k):
            xlabel.append('x%s'%i)
            
    assert (k == len(xlabel)) 
    
    f = open(fname, "w+")
    
    f.write(('\\small \n'+
            '\\begin{tabular}{lclc} \n'+
            '\\hline \n'+
            '\\textbf{Dep. Variable:} & %s & \\textbf{  R-squared: } &  %s \\\\ \n'%(ylabel, np.round(r2,3))  ))
    
    f.write(('\\textbf{No. Observations:} & %s & \\textbf{ AIC:} & %s \\\\ \n'%(nobs,aic)+
                                                                                    
            '\end{tabular} \n'))
    
    
    f.write('\n\\begin{tabular}{lcccc} \n')
    for comp in range(ncomp):
        f.write('\\hline \n')
        f.write('\\textbf{Phase %s} & \\textbf{Est.} & \\textbf{Std. Err.} &'%(1+comp)+ 
                '\\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\ \n')
        f.write('\\hline \\\\ \n')
        
        #isolate params
        comp_params = params[comp]
        comp_se = se[comp]
        comp_t = comp_params[:-1]/comp_se
        comp_p = 1 - stats.t.cdf(np.abs(comp_t),df=(nobs-k)) + stats.t.cdf(-np.abs(comp_t),df=(nobs-k))
        
        #round everything
        comp_params = np.round(comp_params,5)
        comp_se = np.round(comp_se,5)
        comp_t = np.round(comp_t,5)
        comp_p = np.round(comp_p,5)
        
        lamb, lamb_se = comp_params[0], comp_se[0]
        #lamb_t, lamb_p = comp_params[0],  comp_t[0], comp_p[0]
        beta, beta_se, beta_t, beta_p = comp_params[1:-1], comp_se[1:], comp_t[1:], comp_p[1:]
        sigma = comp_params[-1]
        
        if ncomp > 1:
            f.write('\\textbf{Pr(phase %s)} & %s  & %s & & \\\\ \\\\ \n'%(comp+1, lamb, lamb_se) )
        
        for i in range(k):
            f.write('\\textbf{%s} & %s & (%s) & %s & %s \\\\ \\\\ \n'%(xlabel[i],beta[i],beta_se[i],
                                                                             beta_t[i],beta_p[i]) )
        
        #f.write('\\textbf{Variance} & %s &  & & \\\\ \\\\ \n'%(sigma) )
    f.write('\\hline \\\\ \n')    
    f.write('\end{tabular} \n')
    f.close()
    
    #print output
    f = open(fname, "r")
    print f.read()
    f.close()

In [17]:
reg1 = pd.read_csv('data/clean_milk1.csv')
print reg1.columns

#variables names
lmilk = ['LWW']
auct_key = ['YEAR','MONTH','DAY','SYSTEM','FMOZONE']
lcts = ['LFMO','LGAS','LPOPUL','LQWW']#,'LMEALS']
dummies = ['COOLER','ESC']

fekeys = list(reg1.columns[15:-9])


lags = 4
lagkeys = [l+str(i) for l in ['LWW_min','LWW_max'] for i in range(1,1+lags)]

bid_key = auct_key + ['VENDOR'] + ['COUNTY']
covariates = lcts + dummies + fekeys
hist = ['INC'] + lagkeys

Index([u'Unnamed: 0', u'YEAR', u'MONTH', u'DAY', u'SYSTEM', u'FMOZONE',
       u'VENDOR', u'COUNTY', u'LWW', u'LFMO', u'LGAS', u'LPOPUL', u'LQWW',
       u'COOLER', u'ESC', u'3', u'6', u'7', u'9', u'INC', u'LWW_min1',
       u'LWW_min2', u'LWW_min3', u'LWW_min4', u'LWW_max1', u'LWW_max2',
       u'LWW_max3', u'LWW_max4'],
      dtype='object')


In [18]:
nice_ww = 'Bids (log-log)'
nice_cov = ['(Intercept)', 'Raw milk', 'Gas',
            'Population', 'Quantity', #'Meals',
            'Cooler', 'Escalated', #+ fekeys
            'Waco','St. Angelo', 'Austin', 'San Antonio']
nice_lags = [l+str(i) for l in ['Min. lag \#','Max. lag \#'] for i in range(1,1+lags)]
nice_lags = ['Incumbency'] + nice_lags

print len(nice_cov)
print len(covariates)

11
10


In [19]:
est0 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),1)
write_table('results/ols_results.tex', est0, labels=(nice_ww, nice_cov))

\small 
\begin{tabular}{lclc} 
\hline 
\textbf{Dep. Variable:} & Bids (log-log) & \textbf{  R-squared: } &  0.158 \\ 
\textbf{No. Observations:} & 3823 & \textbf{ AIC:} & -8399.7 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\hline 
\textbf{Phase 1} & \textbf{Est.} & \textbf{Std. Err.} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\hline \\ 
\textbf{(Intercept)} & -2.32583 & (0.07014) & -33.15816 & 0.0 \\ \\ 
\textbf{Raw milk} & 0.18546 & (0.02613) & 7.09658 & 0.0 \\ \\ 
\textbf{Gas} & 0.01672 & (0.00401) & 4.16971 & 3e-05 \\ \\ 
\textbf{Population} & 0.00709 & (0.00184) & 3.84354 & 0.00012 \\ \\ 
\textbf{Quantity} & -0.00354 & (0.002) & -1.77142 & 0.07657 \\ \\ 
\textbf{Cooler} & 0.01847 & (0.00292) & 6.32972 & 0.0 \\ \\ 
\textbf{Escalated} & -0.0229 & (0.00274) & -8.34616 & 0.0 \\ \\ 
\textbf{Waco} & -0.06712 & (0.00412) & -16.30108 & 0.0 \\ \\ 
\textbf{St. Angelo} & -0.05418 & (0.00807) & -6.71672 & 0.0 \\ \\ 
\textbf{Austin} & -0.08556 & (0.01378) & -6.2108 & 0.0 \\ \\ 
\textbf{San Anto

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [20]:
est1 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates + hist]),1)
write_table('results/hist_results.tex', est1, labels=(nice_ww, nice_cov + nice_lags))

\small 
\begin{tabular}{lclc} 
\hline 
\textbf{Dep. Variable:} & Bids (log-log) & \textbf{  R-squared: } &  0.195 \\ 
\textbf{No. Observations:} & 3823 & \textbf{ AIC:} & -8552.5 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\hline 
\textbf{Phase 1} & \textbf{Est.} & \textbf{Std. Err.} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\hline \\ 
\textbf{(Intercept)} & -1.5375 & (0.09889) & -15.54688 & 0.0 \\ \\ 
\textbf{Raw milk} & 0.1245 & (0.02614) & 4.76353 & 0.0 \\ \\ 
\textbf{Gas} & 0.01666 & (0.00396) & 4.20662 & 3e-05 \\ \\ 
\textbf{Population} & 0.00376 & (0.00185) & 2.02919 & 0.04251 \\ \\ 
\textbf{Quantity} & -0.00135 & (0.00199) & -0.67705 & 0.49842 \\ \\ 
\textbf{Cooler} & 0.01833 & (0.00287) & 6.38353 & 0.0 \\ \\ 
\textbf{Escalated} & -0.0219 & (0.00269) & -8.13833 & 0.0 \\ \\ 
\textbf{Waco} & -0.06922 & (0.00404) & -17.12198 & 0.0 \\ \\ 
\textbf{St. Angelo} & -0.05885 & (0.00793) & -7.42203 & 0.0 \\ \\ 
\textbf{Austin} & -0.09643 & (0.01352) & -7.13092 & 0.0 \\ \\ 
\textbf{San Ant

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in divide


In [21]:
est2 = estimate(reg1['LWW'],sm.add_constant(reg1[covariates]),2)
write_table('results/prelim_results.tex', est2, labels=(nice_ww, nice_cov))

\small 
\begin{tabular}{lclc} 
\hline 
\textbf{Dep. Variable:} & Bids (log-log) & \textbf{  R-squared: } &  0.41 \\ 
\textbf{No. Observations:} & 3823 & \textbf{ AIC:} & -9004.1 \\ 
\end{tabular} 

\begin{tabular}{lcccc} 
\hline 
\textbf{Phase 1} & \textbf{Est.} & \textbf{Std. Err.} &\textbf{t} & \textbf{P $>$ $|$ t $|$} \\ 
\hline \\ 
\textbf{Pr(phase 1)} & 0.31328  & 0.25513 & & \\ \\ 
\textbf{(Intercept)} & -2.72138 & (0.05916) & -46.00008 & 0.0 \\ \\ 
\textbf{Raw milk} & 0.34896 & (0.02199) & 15.87006 & 0.0 \\ \\ 
\textbf{Gas} & 0.00668 & (0.00349) & 1.91691 & 0.05532 \\ \\ 
\textbf{Population} & -0.00347 & (0.00168) & -2.06565 & 0.03893 \\ \\ 
\textbf{Quantity} & 0.00327 & (0.00184) & 1.77861 & 0.07538 \\ \\ 
\textbf{Cooler} & 0.00408 & (0.00247) & 1.654 & 0.09821 \\ \\ 
\textbf{Escalated} & -0.00487 & (0.00225) & -2.1645 & 0.03049 \\ \\ 
\textbf{Waco} & -0.17572 & (0.00362) & -48.55432 & 0.0 \\ \\ 
\textbf{St. Angelo} & -0.13892 & (0.00681) & -20.39945 & 0.0 \\ \\ 
\textbf{Austin

In [24]:
test1 = estimate(Y, sm.add_constant(X), 2)
test2 = estimate(Y, sm.add_constant(X), 1)

def nonnested_test(model1,model2):
    """test for non nested models quang vuong"""
    
    params1, se1, r21, y1, x1, ncomp1, ll1 = model1
    params2, se2, r22, y2, x2, ncomp2, ll2 = model2
    nobs, k = x1.shape
    
    k1 = params1.shape[1]*ncomp1 - 1 
    k2 = params2.shape[1]*ncomp2 - 1
    
    var1 = (ll1 -ll2).std()
    test1 = (ll1.sum() - ll2.sum() - k1 + k2)*nobs**(-.5)
    test1 = test1/var1
    p1 = 1 - stats.t.cdf(np.abs(test1),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test1),df=(nobs-k1-k2))
    
    var2 =  ((ll1 - ll2)**2).mean()**.5
    test2 = (ll1.sum() - ll2.sum() - k1 + k2 )*nobs**(-.5)
    test2 = test2/var2
    p2 = 1 - stats.t.cdf(np.abs(test2),df=(nobs-k1-k2)) + stats.t.cdf(-np.abs(test2),df=(nobs-k1-k2))
    
    return test1, test2, p1, p2

print nonnested_test(est2,est1)

(7.690497444933137, 7.629614031792577, 1.8612577223820765e-14, 2.9688370983063555e-14)


In [25]:
def write_nonnested(model1,model2,fname):
    test1, test2, p1, p2 = nonnested_test(model1,model2)
    test1, test2, p1, p2 = np.round(test1,5), np.round(test2,5), np.round(p1,5), np.round(p2,5) 
    f = open(fname, "w+")
    f.write('\\begin{tabular}{lcc}')
    f.write('\n\\hline \n & \\textbf{t} & \\textbf{P $>$ $|$ t $|$} \\\\')
    f.write('\n\\hline')
    f.write('\n\\textbf{Test 1} & %s & %s \\\\'%(test1,p1))
    f.write('\n\\textbf{Test 2} & %s & %s \\\\'%(test2,p2))
    f.write('\\hline \\\\ \n')   
    f.write('\n\\end{tabular}\n')
    f.close()

write_nonnested(est2,est1,'results/test_stat.tex')